## Importing necessary libraries

In [6]:
import nltk
import re
import sqlite3
import tensorflow as tf
import pandas as pd
from string import punctuation
from keras.preprocessing import text
from urllib.parse import unquote
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader
physical_devices = tf.config.list_physical_devices('GPU')
# physical_devices

# "../example.csv"

## Creating the corpus of words


In [2]:
df = pd.read_csv("../combined_logs.csv")
print(df['requested_url'][0])
conn = sqlite3.connect('data.db')
df.to_sql('data', conn, index=False, if_exists='replace')
corpus_path = "corpus"
url_tokens = []

chunk_size = 10000
for chunk in pd.read_sql_query('SELECT requested_url FROM data', conn, chunksize=chunk_size):
    urls = chunk['requested_url']
    for url in urls:
        decoded_url = unquote(url)
        tokens = word_tokenize(decoded_url)
        url_tokens.extend(tokens)

conn.close()

corpus = PlaintextCorpusReader(corpus_path, '.*')

corpus_raw = ' '.join(url_tokens)
with open(corpus_path + "/url_corpus.txt", "w", encoding="utf-8") as f:
    f.write(corpus_raw)

print("Corpus created successfully.")

/fr/pag492/explora-platges-i-ports-2/id12/les-anquines.htm
Corpus created successfully.


## Preprocess our corpus of URLs

In [7]:
def preprocess_url(url):
    url = re.sub(r'[/.-]', ' ', url)
    tokens = word_tokenize(url)
    return tokens

preprocessed_tokens = []
for url in url_tokens:
    preprocessed_tokens.extend(preprocess_url(url))

preprocessed_corpus = ' '.join(preprocessed_tokens)

with open(corpus_path + "/preprocessed_corpus.txt", "w", encoding="utf-8") as f:
    f.write(preprocessed_corpus)

print("Corpus preprocessed successfully.")

Corpus preprocessed successfully.


In [15]:
file_path = "corpus/preprocessed_corpus.txt"

with open(file_path, "r", encoding="utf-8") as file:
    url_text_pp = file.read()

print('Total number of characters in the corpus:', len(url_text_pp))
print('\nProcessed sample:', url_text_pp[:99])

# Tokenize the preprocessed text into individual words
url_words = word_tokenize(url_text_pp)

print(url_words[5])

Total number of characters in the corpus: 50121719

Processed sample: fr pag492 explora platges i ports 2 id12 les anquines htm fr pag492 explora platges i ports 2 id12 
ports


In [18]:
unique_words = list(set(url_words))
unique_words.sort()
vocab_size = len(word_ids) + 1
word_ids = [(word, idx + 1) for idx, word in enumerate(unique_words)]

print(word_ids)

[('!', 1), ('#', 2), ('$', 3), ('&', 4), ("'", 5), ("'Le", 6), ("'s", 7), ('(', 8), (')', 9), ('*', 10), ('+CSCOE+', 11), ('+config', 12), ('+t56S5CX8qh', 13), (',', 14), ('0', 15), ('000', 16), ('0001', 17), ('0004', 18), ('0006', 19), ('0009', 20), ('001', 21), ('0011', 22), ('0012', 23), ('0015', 24), ('0016', 25), ('0019', 26), ('001asesoramientohoteles', 27), ('002', 28), ('0021', 29), ('0028', 30), ('002visitasinspeccion', 31), ('003', 32), ('0035', 33), ('003contactosprofesionales', 34), ('004', 35), ('0040', 36), ('00428a3280d2', 37), ('0044', 38), ('0048', 39), ('004colabpresentaciones', 40), ('005', 41), ('0052', 42), ('0053', 43), ('0059', 44), ('005tramitmunicipales', 45), ('006', 46), ('0060', 47), ('006157', 48), ('0066', 49), ('006materialpromocional', 50), ('007', 51), ('007211', 52), ('0075', 53), ('0077', 54), ('0078', 55), ('008', 56), ('0083', 57), ('009', 58), ('0090', 59), ('00AA001ACF42', 60), ('01', 61), ('01015f000965', 62), ('0109', 63), ('011931', 64), ('0120

In [20]:
# generate skip-grams
skip_grams = [tf.keras.preprocessing.sequence.skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in word_ids]
# view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          pairs[i][0], pairs[i][0], 
          pairs[i][1], pairs[i][1], 
          labels[i]))

ValueError: Unknown format code 's' for object of type 'int'

In [3]:
#print(df.head())
# urls = df['requested_url'][:100]
# decoded_urls = [unquote(url) for url in urls]

# create the corpus
# corpus = " ".join(decoded_urls)

# print(corpus)

In [4]:
!python --version

Python 3.9.18


In [5]:
!pip list

Package                      Version
---------------------------- ---------------
absl-py                      2.1.0
anyio                        4.2.0
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.0.5
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        23.2.0
Babel                        2.14.0
backcall                     0.2.0
beautifulsoup4               4.12.3
bleach                       6.1.0
cachetools                   5.3.2
certifi                      2023.11.17
cffi                         1.16.0
charset-normalizer           3.3.2
click                        8.1.7
colorama                     0.4.6
comm                         0.2.1
contourpy                    1.2.0
cycler                       0.12.1
debugpy                      1.6.7
decorator                    5.1.1
defusedxml                   0.7.1
exceptiongroup               1.


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
